In [27]:
type Monkey = {
    id: int
    items: int seq
    op: int -> int
    test: int
    ifTrue: int
    ifFalse: int
} with
    static member Empty = 
        { 
            id = -1
            items = Seq.empty
            op = id
            test = -1
            ifTrue = -1
            ifFalse = -1
        }
module Monkey =
    open System.Text.RegularExpressions

    let (|MonkeyId|_|) (line: string) =
        let monkeyId = Regex(@"Monkey\s(?<monkey_id>\d*):")
        let isMatch = line |> monkeyId.IsMatch
        if isMatch then
            monkeyId.Matches(line)
            |> Seq.map(fun m -> m.Groups.["monkey_id"].Value)
            |> Seq.tryHead
            |> Option.map int
        else
        None

    let (|StartingItems|_|) (line: string) =
        let startingItems = Regex(@"\d+")
        if line.Contains("Starting items") then 
            startingItems.Matches(line)
            |> Seq.map(fun m -> m.Groups.Values |> Seq.map (fun g -> g.Value))
            |> Seq.concat
            |> Seq.map int
            |> Some
        else 
            None

    let buildOperation (term1:string, op:string, term2:string) =
        let operator =
            match op with 
            | "+" -> (fun a b -> a + b)
            | "*" -> (fun a b -> a * b)
            | _ -> (fun a b -> -1)
        
        match term1, term2 with
        | "old", "old" -> (fun it -> operator it it)
        | "old", aNum -> (fun it -> operator it (aNum |> int) )
        | aNum, "old" -> (fun it -> operator (aNum |> int) it)
        | _ -> id

    let (|Operation|_|) (line:string) =
        let operation = Regex(@"(?<term1>old|\d*) (?<op>\*|\+) (?<term2>old|\d*)")
        if line.Contains("Operation:") then 
            operation.Matches(line)
            |> Seq.map(fun m ->
                m.Groups.["term1"].Value , m.Groups.["op"].Value, m.Groups.["term2"].Value )
            |> Seq.tryHead
            |> Option.map buildOperation
        else 
            None
    let (|Test|_|) (line:string) =
        let test = Regex(@"(\d+)")    
        if line.Contains("Test: divisible by") then
            test.Matches(line)
            |> Seq.map(fun m -> m.Groups.Values |> Seq.map (fun g -> g.Value ))
            |> Seq.concat
            |> Seq.tryHead
            |> Option.map int
        else 
            None

    let outcome outcome (line:string) =
        let test = Regex(@"(\d+)")  
        if line.Contains($"If {outcome}: throw to monkey") then 
            test.Matches(line)
            |> Seq.map(fun m -> m.Groups.Values |> Seq.map (fun g -> g.Value ))
            |> Seq.concat
            |> Seq.tryHead
            |> Option.map int
        else 
            None

    let (|OutcomeTrue|_|) = outcome "true"

    let (|OutcomeFalse|_|) = outcome "false"

    let builder monkey line =
        match line with
        | MonkeyId mId -> { monkey with id = mId }
        | StartingItems items -> { monkey with items = items }
        | Operation opr -> { monkey with op = opr }
        | Test test -> { monkey with test = test }
        | OutcomeTrue mId -> { monkey with ifTrue = mId }
        | OutcomeFalse mId -> { monkey with ifFalse = mId }
        | _ -> monkey

type MonkeyMap = Map<int,Monkey>
module KeepAway =
    let inspect (monkey:Monkey) =
        let inspected =
            monkey.items
            |> Seq.map monkey.op
            |> Seq.map (fun it -> (it / 3))
        { monkey with items = inspected }

    let target item (monkey:Monkey) =
        if item % monkey.test = 0 then
            monkey.ifTrue
        else
            monkey.ifFalse

    let throw (monkeys:MonkeyMap) (item:int, monkeyId:int) =
        monkeys
        |> Map.change monkeyId (fun monkey ->
            match monkey with
            | Some m -> 
                display $"${ item } -> monkey ${ monkeyId }"
                { m with items = [item] |> Seq.append m.items } 
                |> Some
            | None -> None)

    let round (monkeys:MonkeyMap) monkey =
        let inspected = monkey |> inspect
        inspected.items
        |> Seq.map (fun it -> it, target it inspected)
        |> Seq.fold throw monkeys
        |> Map.change monkey.id (fun it -> match it with Some m -> { m with items = Seq.empty } |> Some | None -> None)



let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadAllLines(ResolutionFolder + "/testcase11.txt")

let monkeys =
    lines
    |>Seq.chunkBySize 7
    |>Seq.map (Seq.fold Monkey.builder Monkey.Empty)
    |>Seq.map (fun monkey -> monkey.id, monkey)
    |>Map.ofSeq

monkeys
|> Map.values
|> Seq.sortBy (fun it -> it.id)
|> Seq.fold KeepAway.round monkeys

id,items,op,test,ifTrue,ifFalse
3,"[ 74, 500 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 3, operator: FSI_0140+MonkeyModule+operator@46-52 }",17,0,1


id,items,op,test,ifTrue,ifFalse
3,"[ 74, 500, 620 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 3, operator: FSI_0140+MonkeyModule+operator@46-52 }",17,0,1


id,items,op,test,ifTrue,ifFalse
0,[ 20 ],"{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 19, operator: FSI_0140+MonkeyModule+operator@47-53 }",23,2,3


id,items,op,test,ifTrue,ifFalse
0,"[ 20, 23 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 19, operator: FSI_0140+MonkeyModule+operator@47-53 }",23,2,3


id,items,op,test,ifTrue,ifFalse
0,"[ 20, 23, 27 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 19, operator: FSI_0140+MonkeyModule+operator@47-53 }",23,2,3


...

id,items,op,test,ifTrue,ifFalse
1,[ 2080 ],"{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 6, operator: FSI_0140+MonkeyModule+operator@46-52 }",19,2,0


id,items,op,test,ifTrue,ifFalse
3,"[ 74, 500, 620, 1200 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 3, operator: FSI_0140+MonkeyModule+operator@46-52 }",17,0,1


id,items,op,test,ifTrue,ifFalse
3,"[ 74, 500, 620, 1200, 3136 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 3, operator: FSI_0140+MonkeyModule+operator@46-52 }",17,0,1


id,items,op,test,ifTrue,ifFalse
1,"[ 2080, 25 ]","{ FSI_0140+MonkeyModule+buildOperation@52-85: term2: 6, operator: FSI_0140+MonkeyModule+operator@46-52 }",19,2,0


key id items op test ifTrue ifFalse 0 0 [ 20, 23, 27, 26 ] term2 operator 19 FSI_0140+MonkeyModule+operator@47-53 23 2 3 1 1 [ 2080, 25 ] term2 operator 6 FSI_0140+MonkeyModule+operator@46-52 19 2 0 2 2 [ ] operator FSI_0140+MonkeyModule+operator@47-53 13 1 3 3 3 [ ] term2 operator 3 FSI_0140+MonkeyModule+operator@46-52 17 0 1